In [150]:
!pip install geopandas
!pip install contextily

In [151]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
   
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

def wait_for_job(job_id: str) -> str:
    return job_status_for(job_id)

In [152]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import pandas as pd
import geopandas as gpd
import backoff
import shapely
import glob
import os

In [153]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [154]:
iso3 = "COD"  ##ARG URY
file_path = f"/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_{iso3}.csv"
tileindex_df = pd.read_csv(file_path)
# Display the first few rows of the DataFrame
#print(tileindex_df.head())

In [ ]:
36810
36811
37171
37174
37355
37706

In [155]:
json_files = tileindex_df['s3path']
AOIs = json_files
print(len(AOIs))
#36806
aoi_name = os.path.basename(AOIs[0]).split('.')[0]
aoi_name

238


'tile_num_34655'

In [98]:
indices = [3,17,114,119,120,121,136,158,176,177,182,187,190,191,192,193,195,196,205,206,209,215,217,221,233]

In [128]:
indices = [124,125,161,164,184,212]

In [23]:
# list(json_files)

In [156]:
username = maap.profile.account_info()["username"]
# username = "abarenblitt"
assert username

In [165]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

# PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
# PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
# PRODUCTS = ["L2A","L4A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"


variables = []
for n in np.arange(0, 30,1):
    variables.append('cover_z' + str(n))
    variables.append('pai_z' + str(n))
    variables.append('pavd_z' + str(n))

AOIs = json_files#[123:126]
# AOIs = [json_files[i] for i in indices if i < len(json_files)]  # Check if index is within bounds

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS:
        out_dir = os.path.join("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles/",iso3)
        out_file = os.path.join(out_dir,f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg")
        if not os.path.exists(out_file):
            aoi_name = os.path.basename(each_aoi).split('.')[0]
            print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
            #aoi_url = each_aoi
            inputs = dict(
               aoi=each_aoi,
               doi="L2A",
               lat="lat_lowestmode",
               lon="lon_lowestmode",
               beams="all",
               columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh100",
               #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
               #columns="all",
               query="quality_flag == 1",
               limit = 75_000,
               #temporal="-",
               output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
            )
            result = maap.submitJob(
                identifier="gedi-subset",
                algo_id="gedi-subset",
                version="0.9.0",
                queue="maap-dps-worker-64gb",
                username=username,
                **inputs
            )
        
            inputs
            job_id = result.id
            job_id or result

        
    if "L2B" in PRODUCTS: 
        out_dir = os.path.join("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2B_tiles/",iso3)
        out_file = os.path.join(out_dir,f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg")
        if not os.path.exists(out_file):
            aoi_name = os.path.basename(each_aoi).split('.')[0]
            print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
            #aoi_url = each_aoi
            inputs = dict(
               aoi=each_aoi,
               doi="L2B",
               lat="geolocation/lat_lowestmode",
               lon="geolocation/lon_lowestmode",
               beams="all",
               columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover,pai,fhd_normal,"+",".join(variables),
               query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
               limit = 75_000,
               #temporal="-",
               output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
            )
            result = maap.submitJob(
                identifier="gedi-subset",
                algo_id="gedi-subset",
                version="0.9.0",
                queue="maap-dps-worker-32gb",
                username=username,
                **inputs
            )
            inputs
            job_id = result.id
            job_id or result

        
    if "L4A" in PRODUCTS: 
        out_dir = os.path.join("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/",iso3)
        out_file = os.path.join(out_dir,f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg")
        if not os.path.exists(out_file):
            aoi_name = os.path.basename(each_aoi).split('.')[0]
            print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
            #aoi_url = each_aoi
            inputs = dict(
               aoi=each_aoi,
               doi="L4A",
               lat="lat_lowestmode",
               lon="lon_lowestmode",
               beams="all",
               columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
               query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
               limit = 75_000,
               #temporal="-",
               output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
            )
            result = maap.submitJob(
                identifier="gedi-subset",
                algo_id="gedi-subset",
                version="0.9.0",
                queue="maap-dps-worker-32gb",
                username=username,
                **inputs
            )
            inputs
            job_id = result.id
            job_id or result

Thanks, I'll run the GEDI L2A subsetter for tile_num_36814!!


In [1]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

#PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
# PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            queue="maap-dps-worker-32gb",
            username=username,
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L2B" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,cover_z,land_cover_data/landsat_treecover,pai,pai_z,fhd_normal,pavd_z",#"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            queue="maap-dps-worker-32gb",
            username=username,
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            queue="maap-dps-worker-32gb",
            username=username,
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result


NameError: name 'json_files' is not defined

In [58]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

# PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
# PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
PRODUCTS = ["L2B","L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            queue="maap-dps-worker-32gb",
            username=username,
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L2B" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,cover_z,land_cover_data/landsat_treecover,pai,pai_z,fhd_normal,pavd_z",#"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            queue="maap-dps-worker-32gb",
            username=username,
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            queue="maap-dps-worker-32gb",
            username=username,
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result


Thanks, I'll run the GEDI L2B subsetter for tile_num_29418!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_29419!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_29598!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_29599!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_29600!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_29778!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_29779!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_29780!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_29958!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_29959!!


In [12]:
##L4A
## if necessary ## re-run failed subsets
#BRA
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_27090.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_27111.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_27291.geojson'
#CHL
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17937.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17944.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_18117.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_18304.geojson'
#COL
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17727.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17897.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_18267.geojson'
#ECU
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_15749.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_15929.geojson'
#SUR
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22224.geojson'
#VEN
each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_20955.geojson'

inputs = dict(
    aoi=each_aoi,
    doi="L4A",
    lat="lat_lowestmode",
    lon="lon_lowestmode",
    beams="all",
    columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
    query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
    limit = 75_000,
    #temporal="-",
    output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset",
    version="0.7.0",
    queue="maap-dps-worker-32gb",
    username="leitoldv",
    **inputs
)
inputs
job_id = result.id
job_id or result

'10bb0853-2843-479a-abdf-d558bd668254'

In [21]:
##L2B
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_16111.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17915.geojson'
each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_18093.geojson'
inputs = dict(
    aoi=each_aoi,
    doi="L2B",
    lat="geolocation/lat_lowestmode",
    lon="geolocation/lon_lowestmode",
    beams="all",
    columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover,pai,fhd_normal",#"+",".join(variables),
    query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
    limit = 75_000,
    #temporal="-",
    output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset",
    version="0.7.0",
    queue="maap-dps-worker-32gb",
    username="leitoldv",
    **inputs
)
inputs
job_id = result.id
job_id or result

'4458c49d-c255-479d-b841-55376581ee9f'

In [77]:
## L4A

In [40]:
list(json_files)

['https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_29600.geojson',
 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_29779.geojson',
 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_29780.geojson',
 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_29781.geojson',
 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_29958.geojson',
 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_29959.geojson',
 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_29960.geojson',
 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_29961.geojson',
 'https://maap-ops-workspace.s3.amazonaw

In [42]:
# Define the folder path containing the output files from subsetting of tiles
# folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles"
folder_path = os.path.join("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/",iso3)


# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))

13


In [41]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.9.0/gedi-subset/2024/11/26/23"  # Can now set name
out_dir = os.path.join("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/",iso3)

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L4A.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
                print(f"Copying {fname}...")
                subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

Copying tile_num_32660_L4A.gpkg...
Copying tile_num_32661_L4A.gpkg...
Copying tile_num_32480_L4A.gpkg...
Copying tile_num_32479_L4A.gpkg...
Copying tile_num_32481_L4A.gpkg...
Copying tile_num_32662_L4A.gpkg...
Copying tile_num_32483_L4A.gpkg...
Copying tile_num_32300_L4A.gpkg...
Copying tile_num_32482_L4A.gpkg...
Copying tile_num_32299_L4A.gpkg...
Copying tile_num_32484_L4A.gpkg...
Copying tile_num_32664_L4A.gpkg...
Copying tile_num_32663_L4A.gpkg...


In [38]:
## L2A

In [144]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = os.path.join("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles/",iso3)


# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


237


In [146]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.9.0/gedi-subset/2024/12/06"  # Can now set name
out_dir = os.path.join("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles/",iso3)


for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L2A.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
                print(f"Copying {fname}...")
                subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

In [ ]:
## L2B

In [51]:
inputs
job_id = result.id
job_id or result


'1a4306d9-a6fe-4a72-ad02-2753dfc8911c'

In [45]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = os.path.join("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2B_tiles/",iso3)

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))

12


In [44]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.9.0/gedi-subset/2024/11/27"  # Can now set name
out_dir = os.path.join("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2B_tiles/",iso3)


for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L2B.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
                print(f"Copying {fname}...")
                subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

Copying tile_num_32660_L2B.gpkg...
Copying tile_num_32661_L2B.gpkg...
Copying tile_num_32480_L2B.gpkg...
Copying tile_num_32662_L2B.gpkg...
Copying tile_num_32484_L2B.gpkg...
Copying tile_num_32479_L2B.gpkg...
Copying tile_num_32482_L2B.gpkg...
Copying tile_num_32481_L2B.gpkg...
Copying tile_num_32299_L2B.gpkg...
Copying tile_num_32483_L2B.gpkg...
Copying tile_num_32663_L2B.gpkg...
Copying tile_num_32664_L2B.gpkg...


In [9]:
r = maap.listJobs("abarenblitt",algo_id="job-gedi-subset", version="0.9.0", get_job_details=False, status = "Failed")
print(r.text)

{"code": 200, "jobs": [{"id": "56c02587-2aca-4154-9414-437160602c4c", "tags": ["gedi-subset", "gedi-subset"]}, {"id": "508fafc6-fc4e-4f0c-9bef-c12038e51732", "tags": ["gedi-subset", "gedi-subset"]}, {"id": "63ff05ab-57c8-48b7-9f66-95598b284fd5", "tags": ["gedi-subset", "gedi-subset"]}, {"id": "4a1a4a95-4faa-4b56-a3a4-1154ab631f3d", "tags": ["gedi-subset", "gedi-subset"]}, {"id": "01cb3d74-3ce5-4256-b209-e62fcb0d5a5e", "tags": ["gedi-subset"]}, {"id": "6339f939-0183-4c8a-8ce0-dd2b75caaf96", "tags": ["gedi-subset"]}, {"id": "1001bb2e-226f-4553-b311-5ff8732510d7", "tags": ["gedi-subset"]}, {"id": "81adb752-6dec-4dea-98ea-eb581e42039b", "tags": ["gedi-subset", "gedi-subset"]}, {"id": "2666d5fc-2230-44cb-a9a6-bbe3145105eb", "tags": ["gedi-subset", "gedi-subset"]}, {"id": "95933d90-8214-414b-8c66-a4f939885622", "tags": ["gedi-subset", "gedi-subset"]}], "message": "success"}



In [8]:
job_id = result.id
job_id or result

NameError: name 'result' is not defined